In [0]:
from pyspark.sql.functions import *

In [0]:
df_hosa = spark.read.parquet("/mnt/bronze/hosa/providers")

df_hosb = spark.read.parquet("/mnt/bronze/hosb/providers")

df_merged = df_hosa.unionByName(df_hosb)
df_merged.createOrReplaceTempView('providers')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.providers(
ProviderID string,
FirstName string,
LastName string,
Specialization string,
DeptID string,
NPI string,
datasource string,
is_quarantined boolean
)
USING DELTA

In [0]:
%sql
truncate table silver.providers

In [0]:
%sql
insert into silver.providers
select distinct ProviderID,
FirstName,
LastName,
Specialization,
DeptID,
cast(NPI as int) NPI,
datasource,
CASE
  WHEN ProviderID is null or DeptID is null then true
  else false
end as is_quarantined
from providers